In [111]:
import sys
sys.path.append('../cfq')

from itertools import *
from operator import *
import pickle
import re

import numpy as np

from data import RaggedArray, CFQDataset

np.set_printoptions(linewidth=150)

In [3]:
input_dir = '/work/yu_gai/cfq/data/cfq'

In [4]:
df = sqlCtx.read.parquet(f'/data/yu_gai/cfq/dataset.parquet/').sort('index').persist()
question = df.rdd.map(lambda r: r['questionPatternModEntities']).collect()
template = df.rdd.map(lambda r: r['questionTemplate']).collect()
sparql = df.rdd.map(lambda r: r['sparqlPatternModEntities']).collect()

In [62]:
dat = np.load(f'{input_dir}/data.npz')
idx = np.load(f'{input_dir}/splits/mcd3.npz')['testIdxs']
tok_vocab = idx2tok, _ = pickle.load(open(f'{input_dir}/tok-vocab.pickle', 'rb'))
tag_vocab = idx2tag, _ = pickle.load(open(f'{input_dir}/tag-vocab.pickle', 'rb'))
typ_vocab = idx2typ, _ = pickle.load(open(f'{input_dir}/typ-vocab.pickle', 'rb'))
dataset = CFQDataset(idx, dat, tok_vocab, tag_vocab, typ_vocab)
getitem = lambda key: dataset[np.where(idx == key)[0].item()]

In [6]:
out = np.load(f'/data/yu_gai/cfq/None_011021_180317/test_dict.npz')

In [67]:
idx_train = np.load(f'{input_dir}/splits/mcd3.npz')['trainIdxs']
ds_train = CFQDataset(idx_train, dat, tok_vocab, tag_vocab, typ_vocab)
get_train = lambda key: ds_train[np.where(idx_train == key)[0].item()]
templs_train = {seq2templ(get_train(index)['seq']) for index in idx_train}

In [70]:
len(templs_err_), len(templs_err_.intersection(templs_train))

(579, 13)

In [73]:
templs_corr_ = set(seq2templ(getitem(index)['seq']) for em, index in zip(out['em'], out['index']) if em)
len(templs_corr_), len(templs_corr_.intersection(templs_train))

(546, 211)

In [79]:
{templ for templ in templs_err_ if templ not in templs_train and templ.startswith('Did')}

{"Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] a [NP_SIMPLE]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] [entity] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] a [NP_SIMPLE] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] [entity] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] a [NP_SIMPLE] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [ent

In [75]:
{templ for templ in templs_corr_ if templ not in templs_train}

{"Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity]",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity]",
 'Did a [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] [VP_SIMPLE] [entity]',
 'Did a [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE] [VP_SIMPLE] [entity]',
 'Did a [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]',
 'Did a [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_S

In [93]:
sum(1 for em, index in zip(out['em'], out['index']) if not em and seq2templ(getitem(index)['seq']) in templs_comm)

1315

In [97]:
sum(1 for em, index in zip(out['em'], out['index']) if not em and seq2templ(getitem(index)['seq']) in templs_comm_train)

24

In [98]:
templs_comm = templs_err_.intersection(templs_corr_)
templs_comm

{"Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity]",
 'Did a [ADJECTIVE_SIMPLE] [NP_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] [entity] , and [VP_SIMPLE] [entity]',
 'Did a [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]',
 "Did a [NP_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]",
 'Did a [NP_SIMPLE] [VP_SIMPLE] [entity]',
 'Did a [NP_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]',
 'Did a [NP_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE] and [VP_SIMPLE] [entity]',
 'Did a [NP_SIMPLE] that [VP_SIMPLE] [entity] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]',
 'Did a [NP_SIMPLE] that [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity] [VP_SIMPLE] [entity]',
 'Did a [NP_SIMPLE] that [VP_SIMPLE] [entity] and was [VP_SIMPLE] by [entity] [VP_SIMPLE] [entity]',
 'Did a [NP_SIMPLE] that [entity] [VP_SIMPLE] [VP_SIMPLE] [entity]',
 'Did a [NP_SIMPLE] that [entity] [VP_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [en

In [96]:
templs_comm_train = templs_comm.intersection(templs_train)
templs_comm_train

{'Was [entity] a [NP_SIMPLE] whose [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity]',
 'Was [entity] a [NP_SIMPLE] whose [ROLE_SIMPLE] was [VP_SIMPLE] by [entity]',
 'Was a [NP_SIMPLE] whose [ROLE_SIMPLE] was [VP_SIMPLE] by [entity] [entity]',
 'What [NP_SIMPLE] [VP_SIMPLE] and was [VP_SIMPLE] by [entity]',
 'What [NP_SIMPLE] was [VP_SIMPLE] by and [VP_SIMPLE] [entity]',
 'What [ROLE_SIMPLE] of [entity] was [VP_SIMPLE] by [entity]',
 'What was a [NP_SIMPLE] whose [ROLE_SIMPLE] was [VP_SIMPLE] by [entity]',
 'Which [NP_SIMPLE] [VP_SIMPLE] and was [VP_SIMPLE] by [entity]',
 'Which [NP_SIMPLE] was [VP_SIMPLE] by and [VP_SIMPLE] [entity]',
 'Which [ROLE_SIMPLE] of [entity] was [VP_SIMPLE] by [entity]',
 'Which [ROLE_SIMPLE] of [entity] was a [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]'}

In [7]:
n = np.array([getitem(index)['n'] for index in out['index']])
m = n * n
indptr = np.hstack([np.zeros(1), m.cumsum()]).astype(int)

In [8]:
def pos_typ(mem, n_mem, src, dst, typ):
    mem = RaggedArray(mem, np.cumsum(np.hstack([[0], n_mem])))
    i, j = np.where(typ)
    rels = []
    for u, t, v in zip(src[i], j, dst[i]):
        for x, y in product(mem[u], mem[v]):
            rels.append(f'{idx2tok[y]} {idx2typ[t]} {idx2tok[x]}')
    print(*sorted(rels), sep='\n')

In [46]:
templs_err = set(template[index] for em, index in zip(out['em'], out['index']) if not em)
len(templs_err), df.rdd.filter(lambda r: r['index'] in idx_train).filter(lambda r: r['questionTemplate'] in templs_err).count()

(1424, 47)

In [45]:
templs_corr = set(template[index] for em, index in zip(out['em'], out['index']) if em)
len(templs_corr), df.rdd.filter(lambda r: r['index'] in idx_train).filter(lambda r: r['questionTemplate'] in templs_corr).count()

(1928, 2781)

In [58]:
templs_err_ = set(seq2templ(getitem(index)['seq']) for em, index in zip(out['em'], out['index']) if not em)
len(templs_err_), df.rdd.filter(lambda r: r['index'] in idx_train).filter(lambda r: r['questionTemplate'] in templs_err_).count()

(579, 217)

In [52]:
seq2str = lambda seq: ' '.join(idx2tag[idx] for idx in seq)
seq2templ = lambda seq: ' '.join(idx2tag[idx] for idx in takewhile(lambda idx: idx2tag[idx] != '{SEP}', seq))

In [107]:
for em, index in zip(out['em'], out['index']):
    if em and seq2templ(getitem(index)['seq']) == "Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity]":
        print(question[index])
        print(template[index])

Did M3 's cinematographer and writer edit and executive produce M0 , M1 , and M2
Did [entity] 's [ROLE_SIMPLE] and [ROLE_SIMPLE] [VP_SIMPLE] and [VP_SIMPLE] [entity] , [entity] , and [entity]
Did M3 's cinematographer direct , write , executive produce , edit , and produce M0 , M1 , and M2
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]
Did M3 's cinematographer edit , direct , produce , executive produce , and write M0 , M1 , and M2
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]
Did M3 's director write , executive produce , direct , and edit M0 , M1 , and M2
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]
Did M2 's editor edit , executive produce , direct , produce , and write M0 and M1
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] , [VP_

Did M3 's editor write , direct , executive produce , and produce M0 , M1 , and M2
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]
Did M3 's writer executive produce , edit , direct , and produce M0 , M1 , and M2
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]
Did M0 's executive producer marry M1
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity]
Did M1 's parent influence M0
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity]
Did M0 's director and writer write M1 , M2 , M3 , M4 , M5 , and M6
Did [entity] 's [ROLE_SIMPLE] and [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [entity] , [entity] , [entity] , [entity] , and [entity]
Did M5 's producer , star , and director found M0 , M1 , M2 , M3 , and M4
Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [entity] , [entity] , [entity] , and [entity]
Did M0 's spou

Did M3 's cinematographer edit , direct , write , and produce M0 , M1 , and M2
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]


In [103]:
for em, index, start, end in islice(zip(out['em'], out['index'], indptr[:-1], indptr[1:]), 2500):
    if not em and seq2templ(getitem(index)['seq']) in templs_comm:
        dat = getitem(index)
        
        print(question[index])
        print(template[index])
        print(' '.join(idx2tok[idx] for idx in dat['mem']))
        isrel = lambda r: all(s not in r for s in ['!=', ' a ', 'ns:people.person.gender', 'ns:people.person.nationality'])
        print(*sorted(filter(isrel, sparql[index].split('\n')[1 : -1])), sep='\n')
        
        templ = ' '.join([idx2tag[idx] for idx in dat['seq']])
#         print(templ, templ in templ_train)
        src, dst = dat['src'], dat['dst']
        typ_true = out['typ_true'][start : end]
        typ_pred = out['typ_pred'][start : end]
#         print(src, dst)
#         print(typ_true.astype(int))
#         print(typ_pred.astype(int))
        
        print('*' * 75)
        pos_typ(dat['mem'], dat['n_mem'], src, dst, typ_true)
        
        print('*' * 75)
        pos_typ(dat['mem'], dat['n_mem'], src, dst, typ_pred)
        
        print('*' * 75)

Did M0 's prequel star M1
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity]
Did M0 's prequel star M1 {SEP} ?x0 {NIL}
?x0 ns:film.film.sequel M0 .
?x0 ns:film.film.starring/ns:film.performance.actor M1
***************************************************************************
?x0 ns:film.film.sequel M0
?x0 ns:film.film.starring/ns:film.performance.actor M1
***************************************************************************
?x0 ns:film.actor.film/ns:film.performance.film M1
?x0 ns:film.film.sequel M0
?x0 ns:film.film.starring/ns:film.performance.actor M1
***************************************************************************
Did M2 's sequel star M0 and M1
Did [entity] 's [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [entity]
Did M2 's sequel star M0 M1 {SEP} ?x0 {NIL}
?x0 ns:film.film.prequel M2 .
?x0 ns:film.film.starring/ns:film.performance.actor M0 .
?x0 ns:film.film.starring/ns:film.performance.actor M1
************************************************************************

***************************************************************************
?x0 ns:film.director.film M6
?x0 ns:influence.influence_node.influenced M0
?x0 ns:influence.influence_node.influenced M1
?x0 ns:influence.influence_node.influenced M2
?x0 ns:influence.influence_node.influenced M3
?x0 ns:influence.influence_node.influenced M4
***************************************************************************
?x0 ns:influence.influence_node.influenced M0
?x0 ns:influence.influence_node.influenced M1
?x0 ns:influence.influence_node.influenced M2
?x0 ns:influence.influence_node.influenced M3
?x0 ns:influence.influence_node.influenced M4
***************************************************************************
Did a Mexican editor of M2 , M3 , and M4 marry M5 and influence M0
Did a [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] , [entity] , and [entity] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity]
Did a Mexican editor of M2 M3 M4 marry M5 and influence M0 {SEP} ?x0 {NIL}
?x0 ns:film.editor

***************************************************************************
?x0 ns:organization.organization_founder.organizations_founded M1
?x0 ns:organization.organization_founder.organizations_founded M2
?x0 ns:organization.organization_founder.organizations_founded M3
?x0 ns:organization.organization_founder.organizations_founded M4
?x0 ns:organization.organization_founder.organizations_founded M5
?x0 ns:organization.organization_founder.organizations_founded M6
?x0 ns:organization.organization_founder.organizations_founded M7
***************************************************************************
?x0 ns:organization.organization_founder.organizations_founded M1
***************************************************************************
Did a cinematographer found M1 and M2 and found M3 , M4 , M5 , M6 , and M7
Did a [NP_SIMPLE] [VP_SIMPLE] [entity] and [entity] and [VP_SIMPLE] [entity] , [entity] , [entity] , [entity] , and [entity]
Did a cinematographer found M1 M2 and found 

***************************************************************************
?x0 ns:organization.organization_founder.organizations_founded M0
?x0 ns:organization.organization_founder.organizations_founded M1
?x0 ns:organization.organization_founder.organizations_founded M2
?x0 ns:organization.organization_founder.organizations_founded M3
?x0 ns:organization.organization_founder.organizations_founded M4
?x0 ns:organization.organization_founder.organizations_founded M5
?x0 ns:people.person.employment_history/ns:business.employment_tenure.company M0
***************************************************************************
?x0 ns:organization.organization_founder.organizations_founded M0
?x0 ns:organization.organization_founder.organizations_founded M1
?x0 ns:organization.organization_founder.organizations_founded M2
?x0 ns:organization.organization_founder.organizations_founded M3
?x0 ns:organization.organization_founder.organizations_founded M4
?x0 ns:organization.organization_founder.

***************************************************************************
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M1
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M2
***************************************************************************
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M1
***************************************************************************
Did a film distributor that M3 and M4 were employed by and M1 and M2 founded employ M5 and M6
Did a [NP_SIMPLE] that [entity] and [entity] were [VP_SIMPLE] by and [entity] and [entity] [VP_SIMPLE] [VP_SIMPLE] [entity] and [entity]
Did a filmdistributor that M3 M4 were employed by and M1 M2 founded employ M5 M6 {SEP} ?x0 {NIL}
?x0 ns:business.employer.employees/ns:business.employment_tenure.person M3 .
?x0 ns:business.employer.employees/ns:business.employment_tenure.person M

***************************************************************************
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M1
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M2
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M3
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M4
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M5
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M6
***************************************************************************
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M1
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film_distributor_relationship.film M2
?x0 ns:film.film_distributor.films_distributed/ns:film.film_film

***************************************************************************
?x0 ns:film.producer.films_executive_produced M0
?x0 ns:film.producer.film|ns:film.production_company.films M2
?x0 ns:people.person.spouse_s/ns:people.marriage.spouse|ns:fictional_universe.fictional_character.married_to/ns:fictional_universe.marriage_of_fictional_characters.spouses ?x1
***************************************************************************
?x0 ns:film.producer.films_executive_produced M0
?x0 ns:people.person.spouse_s/ns:people.marriage.spouse|ns:fictional_universe.fictional_character.married_to/ns:fictional_universe.marriage_of_fictional_characters.spouses ?x1
***************************************************************************
Did a film star M1 , M2 , and M3
Did a [NP_SIMPLE] [VP_SIMPLE] [entity] , [entity] , and [entity]
Did a film star M1 M2 M3 {SEP} ?x0 {NIL}
?x0 ns:film.film.starring/ns:film.performance.actor M1 .
?x0 ns:film.film.starring/ns:film.performance.actor M2 .
?x0 ns:

In [254]:
[templ for templ in templ_train if templ.startswith("Did a [ADJECTIVE_SIMPLE]")]

[]

In [168]:
ts = {' '.join([idx2tag[idx] for idx in getitem(index)['seq']]) for em, index in zip(out['em'], out['index']) if not em}

In [144]:
idx_train

array([166986, 239216, 177275, ..., 183022, 185019, 139419])

In [219]:
both = templ_train.intersection(templ_test)
print(len(templ_train), len(templ_test), len(both))
list(islice(both, 10))

9148 1058 210


["Who was a [NP_SIMPLE] 's [ROLE_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 'Which [ADJECTIVE_SIMPLE] [NP_SIMPLE] was [VP_SIMPLE] by [entity] and was [VP_SIMPLE] by [entity] {SEP} ?x0 {NIL}',
 'Which [ROLE_SIMPLE] of a [NP_SIMPLE] was [VP_SIMPLE] by [entity] {SEP} ?x0 ?x1 {NIL}',
 'What did [entity] [VP_SIMPLE] and [entity] [VP_SIMPLE] {SEP} ?x0 {NIL}',
 'Which [NP_SIMPLE] was [VP_SIMPLE] by and [VP_SIMPLE] by [entity] {SEP} ?x0 {NIL}',
 "Which [ROLE_SIMPLE] of [entity] was [entity] 's [ROLE_SIMPLE] {SEP} ?x0 {NIL}",
 "Which [NP_SIMPLE] was a [NP_SIMPLE] 's [ROLE_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 'What [ADJECTIVE_SIMPLE] [NP_SIMPLE] was a [ROLE_SIMPLE] of [entity] {SEP} ?x0 {NIL}',
 "What [ROLE_SIMPLE] of [entity] was [entity] 's [ROLE_SIMPLE] {SEP} ?x0 {NIL}",
 'Which [ROLE_SIMPLE] of [entity] [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}']

In [268]:
import re
from operator import *

In [112]:
def n_var(r):
    return len(set(re.findall(r'\?x\d', r['sparqlPatternModEntities'])))

def n(r):
    return sum(1 for tok in r['questionTemplate'].split(' ') if tok in ['[NP_SIMPLE]', '[ROLE_SIMPLE]', 'Who'])

# n_var(df.rdd.take(1)[0])
# n(df.rdd.take(1)[0])
df.rdd.map(lambda r: n_var(r) == n(r)).reduce(add)

115974

In [273]:
df.rdd.filter(lambda r: n_var(r) != n(r)).map(lambda r: r['questionTemplate']).take(10)

["Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [entity] and [VP_SIMPLE] a [NP_SIMPLE] 's [ROLE_SIMPLE]",
 "Did [entity] [VP_SIMPLE] [entity] 's [ROLE_SIMPLE] , [VP_SIMPLE] a [ROLE_SIMPLE] and [ROLE_SIMPLE] of [entity] and [entity] , and

In [ ]:
df.rdd.map(lambda r: r['questionTemplate']).filter(lambda r: r.startswith(''))

In [270]:
df.count()

239357

In [261]:
print(df.rdd.map(lambda r: r['sparqlPatternModEntities']).take(1)[0])

SELECT count(*) WHERE {
?x0 ns:film.actor.film/ns:film.performance.character M1 .
?x0 ns:film.editor.film M0 .
?x0 ns:film.producer.film|ns:film.production_company.films M0 .
?x0 ns:people.person.gender ns:m.02zsn
}


In [255]:
df.columns

['complexityMeasures',
 'expectedResponse',
 'expectedResponseWithMids',
 'index',
 'question',
 'questionPatternModEntities',
 'questionTemplate',
 'questionWithBrackets',
 'questionWithMids',
 'ruleIds',
 'ruleTree',
 'sparql',
 'sparqlPattern',
 'sparqlPatternModEntities']

In [220]:
sum(1 for index in idx_test if ' '.join([idx2tag[idx] for idx in get_test(index)['seq']]) in both)

3081

In [247]:
[x for x in templ_train if x.startswith("Was [entity] 's")]

["Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 ?x2 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 ?x2 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Was [entity] 's [

In [249]:
[x for x in templ_train if "a [NP_SIMPLE]" in x]

["Who was [VP_SIMPLE] by [entity] 's [ROLE_SIMPLE] and [VP_SIMPLE] by a [ROLE_SIMPLE] of a [NP_SIMPLE] {SEP} ?x0 ?x1 ?x2 ?x3 {NIL}",
 'What was [VP_SIMPLE] by , [VP_SIMPLE] by , [VP_SIMPLE] by , [VP_SIMPLE] by , and [VP_SIMPLE] by a [NP_SIMPLE] that [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}',
 'What was [VP_SIMPLE] by a [NP_SIMPLE] that [entity] was [VP_SIMPLE] by and [VP_SIMPLE] by and [VP_SIMPLE] by [entity] {SEP} ?x0 ?x1 {NIL}',
 'Which [NP_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE] that [entity] was [VP_SIMPLE] by and [entity] [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}',
 'Did [entity] [VP_SIMPLE] [entity] , [VP_SIMPLE] a [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] , and [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}',
 "Was [entity] a [NP_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] {SEP} ?x0 ?x1 ?x2 {NIL}",
 'Who was [VP_SIMPLE] by a [NP_SIMPLE] and was [VP_SIMPLE] by a [ROLE_SIMPLE] of a [ROLE_SIMPLE] of [entity] {SEP} ?x0 ?x1 ?x2 ?x3 {NIL}',
 'What [NP_SIMP

In [241]:
templ_test.difference(templ_train)

{"Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] [entity] , and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] a [NP_SIMPLE] , and [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP

In [243]:
set.difference?

In [120]:
for r in df.rdd.filter(lambda r: n_var(r) > 1).take(10):
    print(r['questionPatternModEntities'])
    print(r['sparqlPatternModEntities'])
    print(117 * '*')

Did M1 employ a spouse of a character
SELECT count(*) WHERE {
?x0 ns:people.person.spouse_s/ns:people.marriage.spouse|ns:fictional_universe.fictional_character.married_to/ns:fictional_universe.marriage_of_fictional_characters.spouses ?x1 .
?x1 a ns:fictional_universe.fictional_character .
FILTER ( ?x0 != ?x1 ) .
M1 ns:business.employer.employees/ns:business.employment_tenure.person ?x0
}
*********************************************************************************************************************
Did M3 's male cinematographer 's Canadian spouse 's spouse marry M0
SELECT count(*) WHERE {
?x0 ns:people.person.spouse_s/ns:people.marriage.spouse|ns:fictional_universe.fictional_character.married_to/ns:fictional_universe.marriage_of_fictional_characters.spouses ?x1 .
?x0 ns:people.person.spouse_s/ns:people.marriage.spouse|ns:fictional_universe.fictional_character.married_to/ns:fictional_universe.marriage_of_fictional_characters.spouses M0 .
?x1 ns:people.person.nationality ns:m.0d060

In [121]:
for r in df.rdd.filter(lambda r: 'of' in r['questionPatternModEntities'] and n_var(r) > 1).take(10):
    print(r['questionPatternModEntities'])
    print(r['sparqlPatternModEntities'])
    print(117 * '*')

Did M1 employ a spouse of a character
SELECT count(*) WHERE {
?x0 ns:people.person.spouse_s/ns:people.marriage.spouse|ns:fictional_universe.fictional_character.married_to/ns:fictional_universe.marriage_of_fictional_characters.spouses ?x1 .
?x1 a ns:fictional_universe.fictional_character .
FILTER ( ?x0 != ?x1 ) .
M1 ns:business.employer.employees/ns:business.employment_tenure.person ?x0
}
*********************************************************************************************************************
Did M3 star M2 's actor , star a cinematographer and art director of M0 and M1 , and star M4
SELECT count(*) WHERE {
?x0 ns:film.cinematographer.film M0 .
?x0 ns:film.cinematographer.film M1 .
?x0 ns:film.film_art_director.films_art_directed M0 .
?x0 ns:film.film_art_director.films_art_directed M1 .
?x1 ns:film.actor.film/ns:film.performance.character M2 .
M3 ns:film.film.starring/ns:film.performance.actor ?x0 .
M3 ns:film.film.starring/ns:film.performance.actor ?x1 .
M3 ns:film.film.st